In [1]:
import json
import time
from torch.utils.data import Dataset
import pickle
import config
import ipdb
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
from torch.utils.data import Dataset
class VQADataset_custom_ans(Dataset):
    """VQA dataset"""

    def __init__(self, coco_pkl_file, ques_ann_path, ans_ann_path, mode):
        """
        Args:
            ques_ann (string): Path to the json file with ques_annotations.
            ans_ann (string): Path to the json file with ans_annotations.
        """
        
        self.coco_details = pickle.load(open(coco_pkl_file, 'rb'))['area_and_intersection']        
        self.questions = json.load(open(ques_ann_path, 'r'))['questions']  ###or self.questions = load_vocab(ques_ann_path)
        self.answers = json.load(open(ans_ann_path, 'r'))['answers']
        self.mode = mode

    def __len__(self):
        return len(self.answers)

    def __getitem__(self, idx):
        """Returns ONE data pair-image,match_coco_objects"""
        
        q = self.questions
        area_inter = self.coco_details
        ans = self.answers
        
        assert q[idx]['image_id'] == area_inter[idx]['image_id'] == ans[idx]['image_id']
         
        classes_img = area_inter[idx]['classes_img']
        percent_area_per_catId_all_inst = area_inter[idx]['percent_area_per_catId_all_inst']
        percent_area_per_catId_max_inst = area_inter[idx]['percent_area_per_catId_max_inst']
        if_intersect_overlap_sq5 = area_inter[idx]['if_intersect_overlap_sq5'] #"if_intersect_overlap_default"
        #if_intersect_overlap_default = area_inter[idx]['if_intersect_overlap_default']
        
        # print('Reading image data')
        img_id = self.answers[idx]['image_id']  ####sanity check: should conincide with self.image_ids[idx]- done - same!
        # print(img_id)

        question_type = self.answers[idx]['question_type']
        multipe_choice_answer = self.answers[idx]['multipe_choice_answer']
        answers = self.answers[idx]['answers']
        question_id = self.answers[idx]['question_id']
        answer_type = self.answers[idx]['answer_type']

        # print('Reading nouns data')
        nouns_q_coco = self.questions[idx]['nouns_q_COCO']
        nouns_ans = self.answers[idx]['ans_match_COCO']
        # print(img_id, nouns_img, nouns_q_coco, nouns_ans )
        
        ans10 = [i['answer'] for i in self.answers[idx]['answers']]
        
        return ans10, classes_img, nouns_q_coco, nouns_ans, question_type, multipe_choice_answer, answers,\
                img_id, answer_type, question_id, percent_area_per_catId_all_inst, if_intersect_overlap_sq5, \
                percent_area_per_catId_max_inst
    

In [3]:
coco_val_pkl = './coco_areas_and_intersection/coco_vqa_val2014.json'
coco_train_pkl = './coco_areas_and_intersection/coco_vqa_train2014.json'

In [5]:
import time
def prep_a_json(dataset, filename, area_thresh, overlap_thresh, all_area_thresh=None):
    start = time.time()
    abcd = []
    file_data = {}
    
    for i in range(len(dataset)):
        ans10, classes_img, nouns_q, nouns_ans, ques_type, multiple, ans, img_id, \
        ans_ty, ques_id,percent_area_per_catId_all_inst,if_intersect_overlap, \
        percent_area_per_catId_max_inst = dataset[i]
        
        classes_img_set = sorted(list(set(classes_img)))


        if len(set(ans10))==1:  #uniform answers
            final_target_list = sorted(list(set(classes_img_set) - set(nouns_q) - set(nouns_ans)))
            for cat_id_q_a in sorted(list(set(nouns_q)|set(nouns_ans))):
                for cat_id in classes_img_set:
                    if (cat_id_q_a,cat_id) in if_intersect_overlap.keys():
                        if if_intersect_overlap[(cat_id_q_a,cat_id)]> overlap_thresh:
                            if cat_id in final_target_list:
                                final_target_list.remove(cat_id)

    #         for cat_id in classes_img_set:
    #             if (percent_area_per_catId_all_inst[cat_id] > area_thresh):
    #                 if cat_id in final_target_list:
    #                     final_target_list.remove(cat_id)

            for cat_id in classes_img_set:
                if all_area_thresh is not None:
                    if (percent_area_per_catId_max_inst[cat_id] > area_thresh and \
                        percent_area_per_catId_all_inst[cat_id] > all_area_thresh ):
                        if cat_id in final_target_list:
                            final_target_list.remove(cat_id)
                else:
                    if (percent_area_per_catId_max_inst[cat_id] > area_thresh):
                        if cat_id in final_target_list:
                            final_target_list.remove(cat_id)    
                    #ipdb.set_trace()

            for obj_class in final_target_list :
                new_i_id = str(img_id).zfill(12) + '_' + str(obj_class).zfill(12)
                abcd.append({"question_type": ques_type, 
                             "multiple_choice_answer": multiple, 
                             "answers": ans,
                             "image_id": new_i_id,
                             "answer_type": ans_ty,
                             "question_id": ques_id})
                
            #if img_id == 427135:
                #print(i, ':', dataset[i], final_target_list)
                #print(set([ans1["answer"] for ans1 in ans]))
                #print([coco_dict[i] for i in final_target_list])
            
    file_data['annotations'] = abcd 
    with open(filename, 'w') as outfile_val1:
        json.dump(file_data, outfile_val1)
    print(time.time()-start)


In [7]:
def final_a_prep(mode, coco_pkl, area_thresh, overlap_thresh, all_area_thresh=None):  # mode='val2014' or 'train2014'  string
    question_path = 'tagged_' + mode + '_questions.json'        ## corresponds to question.json file
    answer_path ='tagged_' + mode + '_answers.json'  
    
    # in case you want to play with thresholds- good idea to store in different folders
    #root_dir = os.path.join('mini_datasets_qa', str(area_thresh)+ '_'+str(overlap_thresh))
    root_dir = config.iv_a_dir
    os.makedirs(root_dir,exist_ok=True)
    res_file = 'v2_mscoco_' + mode + '_annotations.json' 
    
    start = time.time()
    dataset_mode = VQADataset_custom_ans(coco_pkl, question_path, answer_path, mode)
    print(time.time()-start)
    prep_a_json(dataset_mode,  os.path.join(root_dir, res_file),area_thresh= area_thresh,\
                overlap_thresh=overlap_thresh, all_area_thresh=all_area_thresh)
    # sample check to make sure
    print(dataset_mode[0])
    with open( os.path.join(root_dir, res_file)) as f:
        edited_answers = json.load(f)['annotations']
    print(len(edited_answers))
    print(edited_answers[0])

In [8]:
final_a_prep('val2014', coco_val_pkl, 0.1, 0.0)  

8.642129898071289
18.825090169906616
(['down', 'down', 'at table', 'skateboard', 'down', 'table', 'down', 'down', 'down', 'down'], [5, 15, 41, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 8, 27, 31, 67, 1, 15, 1], [1], [41, 41, 42, 36, 41, 41, 67], 'none of the above', 'down', [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}], 262148, 'other', 262148000, {1: 0.099, 5: 0.0, 8: 0.011, 15: 0.077, 27: 0.

In [9]:
final_a_prep('train2014',  coco_train_pkl, 0.1,0.0)  

19.05996012687683
41.754278898239136
(['net', 'net', 'net', 'netting', 'net', 'net', 'mesh', 'net', 'net', 'net'], [37, 1, 40], [58], [], 'what is this', 'net', [{'answer': 'net', 'answer_confidence': 'maybe', 'answer_id': 1}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'netting', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'mesh', 'answer_confidence': 'maybe', 'answer_id': 7}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'net', 'answer_confidence': 'yes', 'answer_id': 10}], 458752, 'other', 458752000, {1: 0.072, 37: 0.0, 40: 0.005}, {(1, 37): 0.011, (1, 40): 0.084, (37, 1): 0.993, (40, 1): 0.991}, {1: 0.072, 37: 0.0, 40: 0.005})
514337
{'question_type': 'what is this'